In [ ]:
#import relevant libraries
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.metrics import explained_variance_score, r2_score, classification_report
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, GroupKFold, GroupShuffleSplit, StratifiedKFold

from scipy import stats
from sklearn.utils import shuffle
from statsmodels.stats.multitest import multipletests as fdr


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# read and clean up ABCD data
# set base dirctories
ABCD_base_dir   = '/Users/elvishadhamala/Documents/yale/ABCD'

#load subj fc data
ABCD_fc= pd.read_csv(os.path.join(ABCD_base_dir, 'ABCD_rsfc_withsubcort_5260subj.csv'), header=None)
ABCD_fc = ABCD_fc.T
ABCD_subj = pd.read_csv(os.path.join(ABCD_base_dir, 'subjects_all_rs_all_score_unrelated_5260.txt'), header=None)
ABCD_fc.insert(0, "subjectkey", ABCD_subj[0], True)
ABCD_fc = ABCD_fc.sort_values(by='subjectkey', ascending=True)

# load subj sex/gender data and site data
ABCD_psychosis = pd.read_csv(os.path.join(ABCD_base_dir, 'abcd_mhy02.csv'), header=0)
ABCD_site = pd.read_csv(os.path.join(ABCD_base_dir, 'abcd_lt01.csv'), header=0)
ABCD_motion = pd.read_csv(os.path.join(ABCD_base_dir, 'score_motion_all_subjects_mf.csv'))

#drop duplicate header rows
header_row = 0
ABCD_psychosis = ABCD_psychosis.drop(header_row)
ABCD_site = ABCD_site.drop(header_row)

In [ ]:
#get site data for relevant subjects
ABCD_site = ABCD_site[ABCD_site.subjectkey.isin(ABCD_fc.subjectkey)]
ABCD_site_y1 = ABCD_site[ABCD_site.eventname == 'baseline_year_1_arm_1']
ABCD_site_y1 = ABCD_site_y1.sort_values(by='subjectkey', ascending=True)
ABCD_site_y1 = ABCD_site_y1[['subjectkey', 'site_id_l']]


In [ ]:
#select subjects for whom we have FC data
ABCD_psychosis = ABCD_psychosis[ABCD_psychosis.subjectkey.isin(ABCD_fc.subjectkey)]
ABCD_psychosis_y1 = ABCD_psychosis[ABCD_psychosis.eventname == 'baseline_year_1_arm_1']
ABCD_psychosis_y1 = ABCD_psychosis_y1.sort_values(by='subjectkey', ascending=True)
ABCD_psychosis_y1 = ABCD_psychosis_y1[['subjectkey', 'sex', 'eventname',
                                       'pps_y_ss_number', 'pps_y_ss_severity_score']]
ABCD_psychosis_y1_sum = ABCD_psychosis_y1.dropna(how='all')



In [ ]:
#clean fc and site data to match gender data
ABCD_fc_clean = ABCD_fc[ABCD_fc.subjectkey.isin(ABCD_psychosis_y1_sum.subjectkey)]
ABCD_site_clean = ABCD_site_y1[ABCD_site_y1.subjectkey.isin(ABCD_psychosis_y1_sum.subjectkey)]

In [ ]:
#sort data by subject key
ABCD_fc_clean = ABCD_fc_clean.sort_values(by='subjectkey', ascending=True)
ABCD_site_clean = ABCD_site_clean.sort_values(by='subjectkey', ascending=True)
ABCD_psychosis_y1_sum = ABCD_psychosis_y1_sum.sort_values(by='subjectkey', ascending=True)

In [ ]:
#load in motion data for the participants
mask = ABCD_motion.subjectkey.isin(ABCD_fc_clean['subjectkey'])
ABCD_motion_incl = ABCD_motion[mask]
ABCD_motion_data = ABCD_motion_incl.sort_values(by='subjectkey', ascending=True)
ABCD_motion_data.reset_index(inplace=True) 
ABCD_motion_data = ABCD_motion_data.drop(columns=['index'])
ABCD_motion_m = ABCD_motion_data[ABCD_motion_data.gender=='M']
ABCD_motion_f = ABCD_motion_data[ABCD_motion_data.gender=='F']

In [ ]:
#cleaned fc, gender, and site data
ABCD_fc_data = ABCD_fc_clean.drop(columns=['subjectkey'])
ABCD_psychosis_data = ABCD_psychosis_y1_sum.drop(columns=['subjectkey'])
ABCD_site_data = ABCD_site_clean.drop(columns=['subjectkey'])

#clean subj data
ABCD_fc_data.reset_index(inplace=True) 
ABCD_psychosis_data.reset_index(inplace=True)
ABCD_site_data.reset_index(inplace=True)

ABCD_fc_data = ABCD_fc_data.drop(columns=['index'])
ABCD_psychosis_data = ABCD_psychosis_data.drop(columns=['index'])
ABCD_site_data = ABCD_site_data.drop(columns=['index'])


In [ ]:
#get sex-specific variables
mask_m = ABCD_psychosis_data.sex=='M'
ABCD_fc_m = ABCD_fc_data[mask_m]
ABCD_psychosis_m = np.double(ABCD_psychosis_data[mask_m].pps_y_ss_severity_score)
ABCD_sex_m = ABCD_psychosis_data[mask_m].sex
ABCD_site_m = ABCD_site_data[mask_m].site_id_l

#get sex-specific variables
mask_f = ABCD_psychosis_data.sex=='F'
ABCD_fc_f = ABCD_fc_data[mask_f]
ABCD_psychosis_f = np.double(ABCD_psychosis_data[mask_f].pps_y_ss_severity_score)
ABCD_sex_f = ABCD_psychosis_data[mask_f].sex
ABCD_site_f = ABCD_site_data[mask_f].site_id_l



In [ ]:
#concatenate values so it's easier to combine and put into the model all together 
#fc = np.concatenate([ABCD_fc_m, ABCD_fc_f])
#psychosis = np.concatenate([ABCD_psychosis_m, ABCD_psychosis_f])
#sex = np.concatenate([ABCD_sex_m, ABCD_sex_f])
#site = np.concatenate([ABCD_site_m, ABCD_site_f])

fc = np.concatenate([ABCD_fc_m])
psychosis = np.concatenate([ABCD_psychosis_m])
sex = np.concatenate([ABCD_sex_m])
site = np.concatenate([ABCD_site_m])


sex[sex=='M']=1
sex[sex=='F']=0

In [ ]:
#exclude any remaining nan
nanmask = np.isnan(psychosis)
psychosis = psychosis[~nanmask]
sex = sex[~nanmask]
sex = np.double(sex)
fc = fc[~nanmask]
site = site[~nanmask]


In [ ]:
results_dir = '/Users/elvishadhamala/Documents/BPM/psychosis_preds/'

#number of repetitions you want to perform
rep = 100
#number of folds you want in the cross-validation
k = 3
#proportion of data you want in your training set and test set
train_size = .66
test_size = 1-train_size

#regression model type
regr = Ridge(max_iter=1000000, solver='lsqr')
#set hyperparameter grid space you want to search through for the model
#adapted from the Thomas Yeo Lab Github: 
#ThomasYeoLab/CBIG/blob/master/stable_projects/predict_phenotypes/He2019_KRDNN/KR_HCP/CBIG_KRDNN_KRR_HCP.m
#alphas = [0, 0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3,
#          3.5, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100, 150, 200, 300, 500, 700, 1000, 2000, 
#          3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

alphas = [0.001, 0.01, 0.1, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100, 150, 200, 300, 500, 700, 1000]

#alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

#param grid set to the hyperparamters you want to search through
paramGrid ={'alpha': alphas}

#set x data to be the input variable you want to use
X = fc
Y = psychosis.T

#number of features 
n_feat = X.shape[1]


In [ ]:
#r^2 - coefficient of determination
r2 = np.zeros([rep])
#explained variance
var = np.zeros([rep])
#correlation between true and predicted (aka prediction accuracy)
corr = np.zeros([rep])
#optimised alpha (hyperparameter)
opt_alpha = np.zeros([rep])
#feature importance extracted from the model
featimp = np.zeros([rep,n_feat])
#for when the feat weights get haufe-inverted
featimp_haufe = np.zeros([rep,n_feat])

In [ ]:
del ABCD_fc
del ABCD_fc_clean


del ABCD_psychosis
del ABCD_psychosis_y1
del ABCD_psychosis_y1_sum

del ABCD_site
del ABCD_site_clean


In [ ]:
for p in range(rep):
    
    #print model # you're on
    print('Model %d' %(p+1))
    
    #if running null models need to shuffle the Y values within site
    #values = Y
    #groups = site

    #for index in np.unique(groups):
    #    mask = groups==index
    #    values[mask] = np.random.permutation(values[mask])
    #Y_shuffle = values
    
    #split into train/test data
    train_inds, test_inds = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state = p).split(X, groups=site))
    
    #set x values based on indices from split
    x_train = X[train_inds]
    x_test = X[test_inds]
        
    #set y values based on indices from split  
    beh_train = Y[train_inds]
    beh_test = Y[test_inds]
    
    #set y values based on indices from split for null predictions  
    #beh_train = Y_shuffle[train_inds]
    #beh_test = Y_shuffle[test_inds]
    
    #set site data based on indices from split
    site_train = site[train_inds]
    site_test = site[test_inds] 
    
    #convert y values to to double
    y_train = np.double(beh_train)
    y_test = np.double(beh_test)

    #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
    best_scores = []
    best_params = []
    
    #set parameters for inner and outer loops for CV
    cv_split = GroupKFold(n_splits=k)
        
    print ("Optimising Models")
            
    #define regressor with grid-search CV for inner loop
    gridSearch = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, verbose=0, cv=cv_split, scoring='explained_variance')

    #fit regressor to the model, use site ID as group category again
    gridSearch.fit(x_train, y_train, groups=site_train)

    #save parameters corresponding to the best score
    best_params.append(list(gridSearch.best_params_.values()))
    best_scores.append(gridSearch.best_score_)
        
    print ("Evaluating Models")
        
    #save optimised alpha values
    opt_alpha[p] = best_params[best_scores.index(np.max(best_scores))][0]

    #for null prediction - use random alpha from set of optimized alphas 
    #rand_alpha = np.random.choice(alphas)
    
    #fit optimized models
    model = Ridge(alpha = opt_alpha[p], max_iter=1000000, solver='lsqr')
    #model = Ridge(alpha = rand_alpha, max_iter=1000000, solver='lsqr')

    model.fit(x_train, y_train);
        
    #evaluate model within sex within behavior
    r2[p]=model.score(x_test,y_test)
    p#rint(r2[p])
        
    preds = []
    preds = model.predict(x_test).ravel()
    
    #compute explained variance 
    var[p] = explained_variance_score(y_test, preds)
    #print(var[p])

    #compute correlation between true and predicted (prediction accuracy)
    corr[p] = np.corrcoef(y_test.ravel(), preds)[1,0]
    #print(corr[p])
    
    print ("Haufe-Transforming Feature Weights")
    
    #extract feature importance
    featimp[p,:] = model.coef_
    
    cov_x = []
    cov_y_pred = []
    
    #generate predictions for y_train 
    y_train_preds = model.predict(x_train).ravel()
    cov_y_pred = np.cov(y_train_preds)
    

    #compute Haufe-inverted feature weights
    cov_x = np.cov(np.transpose(x_train))
    
    #perform matrix multiplication of cov x, feat weights, and divide by cov of predicted y_train
    featimp_haufe[p,:] = featimp_haufe[p,:]*(cov_x)/cov_y_pred

    #save all outputs
    np.save((results_dir + 'fc_r2_sev_m.npy'),r2)
    np.save((results_dir + 'fc_var_sev_m.npy'),var)
    np.save((results_dir + 'fc_corr_sev_m.npy'),corr)
    np.save((results_dir + 'fc_alpha_sev_m.npy'),opt_alpha)
    np.save((results_dir + 'fc_featimp_sev_m.npy'),featimp)
    np.save((results_dir + 'fc_featimp_haufe_sev_m.npy'),featimp_haufe)